In [13]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Convolution1D, Dropout
from keras.optimizers import SGD
from keras.utils import np_utils
import scipy.io as sio

# 读取数据保存到mat
# data = pd.read_csv('./data.csv')
# data = np.array(data)
# label = pd.read_csv('./label.csv')
# label = np.array(label)

# sio.savemat("data.mat", {"data": data} )
# sio.savemat("label.mat", {"label": label} )
# print(data.shape)
# print(label.shape)

# 直接从保存的mat中读取数据
train = sio.loadmat("data.mat")['data']
label = sio.loadmat("label.mat")['label']
# print(train.shape)
# print(label.shape)

# 处理训练集中的NaN，空值
df = pd.DataFrame(train.T)
# print(np.where(np.isnan(df)))
data_without_NaN =df.dropna(axis=1)
train = np.array(data_without_NaN.T)
# 处理label中的NaN，空值
label = np.delete(label, [7583, 7584, 15664, 15665], axis = 0)
# print(train.shape)
# print(label.shape)
  
# 标准化训练集
scaler = StandardScaler().fit(train)
scaled_train = scaler.transform(train)

# 交叉验证分类
sss = StratifiedShuffleSplit(test_size=0.25, random_state=0)
for train_index, valid_index in sss.split(scaled_train, label):
    # print("TRAIN:", train_index, "TEST:", valid_index)
    X_train, X_valid = scaled_train[train_index], scaled_train[valid_index]
    y_train, y_valid = label[train_index], label[valid_index]
    
nb_features = 651 # 每一个特征数量 (Ⅱ, Ⅲ, aVF)   
nb_class = 2 # 最后结果

# 训练集维度变换
X_train_r = np.zeros((len(X_train), nb_features, 3))
X_train_r[:, :, 0] = X_train[:, :nb_features] #0~650
X_train_r[:, :, 1] = X_train[:, nb_features:nb_features*2]# 651~1301
X_train_r[:, :, 2] = X_train[:, nb_features*2:]#1302~end

# 验证集维度变换
X_valid_r = np.zeros((len(X_valid), nb_features, 3))
X_valid_r[:, :, 0] = X_valid[:, :nb_features]
X_valid_r[:, :, 1] = X_valid[:, nb_features:nb_features*2]
X_valid_r[:, :, 2] = X_valid[:, nb_features*2:]

# Keras model 一维卷积
# 不好的是卷积层和滤波器层数较大
# 不能给很好的正确率
model = Sequential()
model.add(Convolution1D(input_shape=(nb_features, 3), kernel_size=1, filters=512))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dropout(0.4))
model.add(Dense(2048, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(nb_class))
model.add(Activation('softmax'))

y_train = np_utils.to_categorical(y_train, nb_class)
y_valid = np_utils.to_categorical(y_valid, nb_class)

sgd = SGD(lr=0.01, nesterov=True, decay=1e-6, momentum=0.9)
model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])

nb_epoch = 1
model.fit(X_train_r, y_train, epochs=nb_epoch, validation_data=(X_valid_r, y_valid), batch_size=16)

Train on 15630 samples, validate on 5210 samples
Epoch 1/1


ResourceExhaustedError: OOM when allocating tensor with shape[333312,2048] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[node training_2/SGD/zeros_2 (defined at D:\Anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:702) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'training_2/SGD/zeros_2', defined at:
  File "D:\Anaconda\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "D:\Anaconda\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "D:\Anaconda\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "D:\Anaconda\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "D:\Anaconda\lib\site-packages\ipykernel\kernelapp.py", line 499, in start
    self.io_loop.start()
  Fil